In [ ]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
import os

In [ ]:
url = None; substance = None; USGS = None; station = None
start_time = "2024-01-01 00:00"
end_time = "2024-12-23 23:00"
input_path = 'Data/'
output_path = 'Data_Extract/Data_Input/'
os.makedirs(output_path, exist_ok=True)
data = pd.read_excel(r"c:\Users\ADMIN\Documents\Zalo Received Files\Book2.xlsx")
data.rename(columns={'Data': 'Substance'}, inplace=True)
data['link'] = data['link'].apply(lambda x: 'https://' + x if not x.startswith('https://') else x)
data.head()

In [ ]:
def read_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        raw_data = response.text
        data_lines = [line for line in raw_data.splitlines() if not line.startswith('#')]
        filtered_data = "\n".join(data_lines)
        df = pd.read_csv(StringIO(filtered_data), sep="\t")
        df = df[[df.columns[2]] + [df.columns[-2]]]
        df.columns = ["Timestamp", "Value"]
        df.drop(index=0, inplace=True)
        print(df.head(5))
        return df
    else:
        print(f"Failed to fetch data. HTTP status code: {response.status_code}")

def append_time(df, start_time, end_time):
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    full_time_range = pd.date_range(start=start_time, end=end_time, freq="H")
    all_data_new = {}

    for timestamp in full_time_range:
        match = df[df['Timestamp'] == timestamp]
        if not match.empty:
            all_data_new[timestamp] = match['Value'].iloc[0]
        else:
            all_data_new[timestamp] = np.nan

    hourly_data_df = pd.DataFrame(list(all_data_new.items()), columns=["Timestamp", "Value"])
    print(hourly_data_df)
    return hourly_data_df

def save_data(df, path):
    df.to_excel(path, index=False)

In [ ]:
for index in data.index:
    if not np.isnan(data.loc[index, 'ID']):
        USGS = int(data.loc[index, 'ID'])
        station = data.loc[index, 'Location']
    if str(data.loc[index, 'link']) != 'nan':
        url = data.loc[index, 'link']
        substance = data.loc[index, 'Substance']
        df = read_url(url)
        df = append_time(df, start_time, end_time)
        path = f"{output_path}/{substance}_{USGS}_{station}.xlsx"
        save_data(df, path)

In [ ]:
import matplotlib.pyplot as plt
df_observed = pd.read_excel(r'f:\Zalo_Receiver\Gage height_14191000_WILLAMETTE RIVER AT SALEM.xlsx')
df_model = pd.read_excel(r'f:\Zalo_Receiver\mucnuoc_14191000.xlsx', skiprows=3)
df_model.columns=['0','1','Value']
def plot_data(df_observed, df_model):
    plt.figure(figsize=(10, 6))
    plt.plot(df_observed["index"], df_observed["Value"], label='Data, USGS 14191000', color='blue')
    plt.plot(df_observed["index"], df_model["Value"], label='Model', color='red', linestyle='--')
    plt.title('Gage Height Over Time')
    plt.xlabel('Date')
    plt.ylabel('Gage Height')
    plt.legend()
    plt.show()

plot_data(df_observed, df_model)

In [ ]:
def calculate_metrics(obs, sim):
    obs = np.array(obs)
    sim = np.array(sim)
    obs_mean = np.nanmean(obs)
    sim_mean = np.nanmean(sim)
    obs_std = np.nanstd(obs)
    sim_std = np.nanstd(sim)
    r2 = 1 - ((sim - obs)**2).sum() / ((obs - obs_mean)**2).sum()
    me = sim_mean - obs_mean
    nse = 1 - ((sim - obs)**2).sum() / ((obs - obs_mean)**2).sum()
    mae = (np.abs(sim - obs)).sum() / len(obs)
    rmse = np.sqrt(((sim - obs)**2).sum() / len(obs))
    return r2, me, nse, mae, rmse

r2, me, nse, mae, rmse = calculate_metrics(df_observed['Value'], df_model['Value'])
print("R2:", r2)
print("ME:", me)
print("NSE:", nse)
print("MAE:", mae)
print("RMSE:", rmse)

In [ ]:
import matplotlib.pyplot as plt
df_observed = pd.read_excel(r'f:\Zalo_Receiver\Temperature_14192015_Willamette River at Keizer.xlsx')
df_model = pd.read_excel(r'f:\Zalo_Receiver\nhietdo2015.xlsx', skiprows=1)
df_model.columns=['No', 'Value']
def plot_data(df_observed, df_model):
    plt.figure(figsize=(10, 6))
    plt.plot(df_observed["Timestamp"], df_observed["Value"], label='Data, USGS 14192015', color='blue')
    plt.plot(df_observed["Timestamp"], df_model["Value"], label='Model', color='red', linestyle='dashed')
    plt.title('Temperature Over Time')
    plt.xlabel('Date')
    plt.ylabel('Temperature')
    plt.legend()
    plt.show()

plot_data(df_observed, df_model)

In [ ]:
def calculate_metrics(obs, sim):
    obs = np.array(obs)
    sim = np.array(sim)
    obs_mean = np.nanmean(obs)
    sim_mean = np.nanmean(sim)
    obs_std = np.nanstd(obs)
    sim_std = np.nanstd(sim)
    r2 = 1 - ((sim - obs)**2).sum() / ((obs - obs_mean)**2).sum()
    me = sim_mean - obs_mean
    nse = 1 - ((sim - obs)**2).sum() / ((obs - obs_mean)**2).sum()
    mae = (np.abs(sim - obs)).sum() / len(obs)
    rmse = np.sqrt(((sim - obs)**2).sum() / len(obs))
    return r2, me, nse, mae, rmse

r2, me, nse, mae, rmse = calculate_metrics(df_observed['Value'], df_model['Value'])
print("R2:", r2)
print("ME:", me)
print("NSE:", nse)
print("MAE:", mae)
print("RMSE:", rmse)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Your lytrinh dictionary
lytrinh = {
    "Clackamas": 216844.1595,
    "Tualatin": 211310.6295,
    "Mollala": 199436.8806,
    "Yamhill": 168973.4942,
    "Santiam": 84493.24208,
    "Luckiamute": 84821.40065,
    "Calapooia": 65798.49166,
    "Marys": 45481.8446,
    "Long_Tom": 19740.75954
}

# Load the data from the Excel file
data = pd.read_excel(r"D:\temp\vebando\minmax_caotrinh_willamette.xlsx")

# Clean the 'Water Level' column by removing certain text
for i in range(len(data['Water Level'])):
    data['Water Level'][i] = data['Water Level'][i].replace('WILLAMETTE ', '')
    data['Water Level'][i] = data['Water Level'][i].replace('WILLAMETTE', '')

data['Water Level'] = data['Water Level'].astype(float)

# Function to plot the data and add arrows for rivers
def plot_data(data):
    plt.figure(figsize=(10, 5))

    # Convert the columns to numpy arrays to ensure they are 1D
    water_levels = data['Water Level'].values
    minimum = data['Minimum'].values
    maximum = data['Maximum'].values

    # Plot the shaded region between the min and max
    plt.fill_between(water_levels, minimum, maximum, color='red', alpha=0.5)
    
    # Plot the min and max as lines
    plt.plot(water_levels, minimum, color='blue')
    plt.plot(water_levels, maximum, color='green')

    # Loop through each river in the lytrinh dictionary
    for river, value in lytrinh.items():
        # Find the closest Water Level to the river's value
        closest_idx = (data['Water Level'] - value).abs().idxmin()
        
        # Use .iloc[] to get the scalar value for the closest match
        water_level_value = data['Water Level'].iloc[closest_idx]
        max_elevation_value = data['Maximum'].iloc[closest_idx]
        
        # Adjust label position to avoid overlap and keep it within the plot bounds
        label_y_offset = 10  # Vertical offset for label

        # Determine dynamic positioning based on space
        if max_elevation_value + label_y_offset > maximum.max():
            label_y_offset = -10  # Adjust position if too high

        # Plot the arrow and river label
        plt.annotate(
            river,  # River name
            xy=(water_level_value, max_elevation_value),  # Position of the arrow
            xytext=(water_level_value, max_elevation_value + label_y_offset),  # Label position above or below
            arrowprops=dict(facecolor='black', arrowstyle='->'),  # Arrow properties
            fontsize=10,
            horizontalalignment='center',
            verticalalignment='bottom' if label_y_offset > 0 else 'top',
            bbox=dict(facecolor='white', edgecolor='none', boxstyle='round,pad=0.2')  # Label box for better readability
        )

    # Adjust x-ticks for better display
    plt.xticks(np.linspace(water_levels.min(), water_levels.max(), 10))
    plt.xlabel('Water Level')
    plt.ylabel('Elevation')
    plt.title('Water Level vs Elevation')

    # Improve layout to avoid cutting off labels or other elements
    plt.tight_layout()

    plt.show()

# Call the function to plot
plot_data(data)
